In [54]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import SnowballStemmer, PorterStemmer, LancasterStemmer
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Load in your data from kaggle.
By working in a kaggle kernel, you can access the data directly from the competition, as well as make your submission without downloading your output file

In [55]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

## Cleaning up Data

In [56]:
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'
train['message'] = train['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)

train['message'] = train['message'].str.lower()

#print(string.punctuation)

def remove_punctuation(message):
    return ''.join([l for l in message if l not in string.punctuation])

train['message'] = train['message'].apply(remove_punctuation)



In [57]:
#pd.set_option('display.max_colwidth', -1)
train.head()


,sentiment,message,tweetid
0,1,polyscimajor epa chief doesnt think carbon dio...,625221
1,1,its not like we lack evidence of anthropogenic...,126103
2,2,rt rawstory researchers say we have three year...,698562
3,1,todayinmaker wired 2016 was a pivotal year in...,573736
4,1,rt soynoviodetodas its 2016 and a racist sexis...,466954


In [58]:
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'
test['message'] = test['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)

test['message'] = test['message'].str.lower()

#print(string.punctuation)

def remove_punctuation(message):
    return ''.join([l for l in message if l not in string.punctuation])

test['message'] = test['message'].apply(remove_punctuation)

In [59]:
test.head()

,message,tweetid
0,europe will now be looking to china to make su...,169760
1,combine this with the polling of staffers re c...,35326
2,the scary unimpeachable evidence that climate ...,224985
3,karoli morgfair osborneink dailykos \nputin go...,476263
4,rt fakewillmoore female orgasms cause global w...,872928


In [60]:
from sklearn.utils import resample
believe = train[train['sentiment'] == 1]
no_belief = train[train['sentiment'] == -1]
neutral = train[train['sentiment'] == 0]
news = train [train['sentiment'] == 2]
class_size = (len(believe)-len(no_belief))/2
new = int(class_size)
#upsample minority
no_belief_upsampled = resample(no_belief,
                              replace=True, # sample with replacement
                              n_samples=len(believe), #match number in majority class
                              random_state=27) #reproducible results
neutral_upsampled = resample(neutral,
                              replace=True, # sample with replacement
                              n_samples=len(believe), #match number in majority class
                              random_state=27) #reproducible results
news_upsampled = resample(news,
                              replace=True, # sample with replacement
                              n_samples=len(believe), #match number in majority class
                              random_state=27) #reproducible results   
#believe_downsampled = resample(believe,
 #                             replace=False, # sample with replacement
  #                            n_samples=new, #match number in majority class
   #                           random_state=27) #reproducible results   
#combine majority and unsampled minority
upsampled = pd.concat([believe, no_belief_upsampled,neutral_upsampled,news_upsampled])             

In [61]:
upsampled.sentiment.value_counts()

-1    8530
 2    8530
 1    8530
 0    8530
Name: sentiment, dtype: int64

In [62]:
train.sentiment.value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

## Splitting out the X variable from the target

In [63]:
y = upsampled['sentiment']
X = upsampled['message']

## Turning text into something your model can read

In [64]:
#vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, stop_words="english")
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, stop_words=None, strip_accents = 'unicode', token_pattern = r'\w{1,}')
X_vectorized = vectorizer.fit_transform(X)

## Splitting the training data into a training and validation set

In [65]:
#X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=.3,shuffle=True, stratify=y, random_state=11)
X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=0.01,shuffle=True, stratify=y, random_state=11)

In [66]:
#from imblearn.over_sampling import SMOTE

#sm = SMOTE(random_state = 2)
#X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

#clf = LogisticRegression()
#model_res = clf.fit(X_train_res, y_train_res)

## Training the model and evaluating using the validation set

In [67]:
rfc = LogisticRegression(solver='newton-cg', C=4.7, random_state=11, fit_intercept=False , multi_class = 'ovr')
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_val)

In [68]:
#logModel = LogisticRegression()

#param_grid = [
 #   {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
  #   'C' : np.logspace(-4, 4, 20),
   #  'solver' : ['lbfgs', 'newton-cg', 'liblinear', 'sag', 'saga'],
    # 'max_iter' : [100, 1000, 2500, 500]
     #}
#]

In [69]:
#rfc = GridSearchCV(logModel, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)

In [70]:
#best_rfc = rfc.fit(X_vectorized,y)

In [71]:
#best_rfc.best_estimator_

## Checking the performance of our model on the validation set

In [72]:
f1_score(y_val, rfc_pred, average="macro")

0.9792781393157556

## Getting our test set ready

In [73]:
testx = test['message']
test_vect = vectorizer.transform(testx)

## Making predictions on the test set and adding a sentiment column to our original test df

In [74]:
y_pred = rfc.predict(test_vect)

In [75]:
test['sentiment'] = y_pred

In [76]:
test.head()

,message,tweetid,sentiment
0,europe will now be looking to china to make su...,169760,1
1,combine this with the polling of staffers re c...,35326,1
2,the scary unimpeachable evidence that climate ...,224985,1
3,karoli morgfair osborneink dailykos \nputin go...,476263,1
4,rt fakewillmoore female orgasms cause global w...,872928,0


## Creating an output csv for submission

In [77]:
test[['tweetid','sentiment']].to_csv('Submission.csv', index=False)